In [1]:
import pandas as pd
import numpy as np
import matplotlib.pylab as plt
%pylab inline
import geopandas as gpd
import zipfile
from sklearn.metrics import cohen_kappa_score
import statsmodels.formula.api as smf
from sklearn.cross_validation import train_test_split
from sklearn.metrics import confusion_matrix

Populating the interactive namespace from numpy and matplotlib


`%matplotlib` prevents importing * from pylab and numpy
/Users/kristikorsberg/devel/venv/lib/python2.7/site-packages/sklearn/cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [2]:
pluto_updated = gpd.read_file('../data_processed/pluto16.shp')

In [3]:
pluto_updated.head(2)

,BBL,geometry
0,5007470028,POLYGON ((-74.14250944979312 40.60980609846242...
1,5006230252,POLYGON ((-74.08780465394567 40.61401983466899...


In [4]:
features = pd.read_csv('../data_processed/pluto_16')

//anaconda/lib/python2.7/site-packages/IPython/core/interactiveshell.py:2723: DtypeWarning: Columns (27,37,40,46,53,64,65,84,85) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [5]:
features.head(2)

,Unnamed: 0,APPBBL,APPDate,Address,AllZoning1,AllZoning2,AreaSource,AssessLand,AssessTot,BBL,...,YearAlter2,YearBuilt,ZMCode,ZipCode,ZoneDist1,ZoneDist2,ZoneDist3,ZoneDist4,ZoneMap,geometry
0,0,0.0,NaN,437 PURDY AVENUE,R3-1,NaN,7,15120.0,23400.0,5.007470e+09,...,0,1960,NaN,10314,R3-1,NaN,NaN,NaN,20d,"POLYGON ((944681.4200000018 161477.381400004, ..."
1,1,0.0,NaN,CUNARD AVENUE,R3A/HS,NaN,4,21518.0,21518.0,5.006230e+09,...,0,0,NaN,10304,R3A,NaN,NaN,NaN,21d,"POLYGON ((959872.0577999949 162992.5866000056,..."


In [6]:
features['AssessSqft'] = features['AssessTot']/features['BldgArea']
features = features[features['YearBuilt']>1750]
features = features[features['AssessLand']> 0]
features = features[features['AssessSqft'] > 0]
features = features[(features['BuiltFAR'] > 0) & (features['BuiltFAR'] < 2088)]
features = features[features['NumBldgs'] > 0]
features = features[features['NumFloors'] > 0]
features = features[features['ResArea'] > 0]
features = features[features['UnitsRes'] < 8000]
features = features[features['UnitsRes'] > 0]

In [7]:
len(features)

756433

In [8]:
pluto_updated.head(2)

,BBL,geometry
0,5007470028,POLYGON ((-74.14250944979312 40.60980609846242...
1,5006230252,POLYGON ((-74.08780465394567 40.61401983466899...


In [9]:
#print(len(pluto_updated))

In [10]:
v = pd.read_csv('../data_processed/VIOLATION_HEATHOTWATER_by_bbl_season.csv')
v = v.drop('Unnamed: 0',axis=1)

In [11]:
features.columns

Index([u'Unnamed: 0', u'APPBBL', u'APPDate', u'Address', u'AllZoning1',
       u'AllZoning2', u'AreaSource', u'AssessLand', u'AssessTot', u'BBL',
       u'BldgArea', u'BldgClass', u'BldgDepth', u'BldgFront', u'Block',
       u'BoroCode', u'Borough', u'BsmtCode', u'BuiltCode', u'BuiltFAR',
       u'CB2010', u'CD', u'CT2010', u'ComArea', u'CommFAR', u'CondoNo',
       u'Council', u'EDesigNum', u'Easements', u'ExemptLand', u'ExemptTot',
       u'Ext', u'FacilFAR', u'FactryArea', u'FireComp', u'GarageArea',
       u'HealthArea', u'HistDist', u'IrrLotCode', u'LandUse', u'Landmark',
       u'Lot', u'LotArea', u'LotDepth', u'LotFront', u'LotType', u'LtdHeight',
       u'MAPPLUTO_F', u'NumBldgs', u'NumFloors', u'OfficeArea', u'OtherArea',
       u'Overlay1', u'Overlay2', u'OwnerName', u'OwnerType', u'PLUTOMapID',
       u'PolicePrct', u'ProxCode', u'ResArea', u'ResidFAR', u'RetailArea',
       u'SHAPE_Area', u'SHAPE_Leng', u'SPDist1', u'SPDist2', u'Sanborn',
       u'SchoolDist', u'SplitZone',

In [14]:
print len(features)
print len(v)

756433
17472


In [15]:
features['ResArea_PerUnit'] = features['ResArea']/features['UnitsRes']

In [16]:
### drop categorical data and anything not pluto
features = features[['BBL', 'NumBldgs', 'NumFloors', 'ResArea', 'YearBuilt', 'AssessSqft', 'ResArea_PerUnit']]

In [17]:
features.columns

Index([u'BBL', u'NumBldgs', u'NumFloors', u'ResArea', u'YearBuilt',
       u'AssessSqft', u'ResArea_PerUnit'],
      dtype='object')

In [18]:
features['BuildingAge'] = 2017-features['YearBuilt']
features = features.drop('YearBuilt',axis=1)

In [19]:
features.head(2)

,BBL,NumBldgs,NumFloors,ResArea,AssessSqft,ResArea_PerUnit,BuildingAge
0,5.007470e+09,1,1.0,1488,15.725806,744.0,57
2,5.007130e+09,2,2.5,1316,17.174772,1316.0,97


In [20]:
print features['BBL'].dtypes
print v['BBL'].dtypes
print pluto_updated['BBL'].dtypes

float64
int64
object


In [21]:
fv = pd.merge(features, v, how='left', on='BBL')

In [22]:
print len(fv)

756433


In [23]:
features.isnull().sum()

BBL                0
NumBldgs           0
NumFloors          0
ResArea            0
AssessSqft         0
ResArea_PerUnit    0
BuildingAge        0
dtype: int64

In [24]:
fv.isnull().sum()

BBL                     0
NumBldgs                0
NumFloors               0
ResArea                 0
AssessSqft              0
ResArea_PerUnit         0
BuildingAge             0
Sum_season_vio     739199
Avg_season_vio     739199
dtype: int64

In [25]:
fv = fv.fillna(value=0.0)

In [26]:
fv.isnull().sum()

BBL                0
NumBldgs           0
NumFloors          0
ResArea            0
AssessSqft         0
ResArea_PerUnit    0
BuildingAge        0
Sum_season_vio     0
Avg_season_vio     0
dtype: int64

In [27]:
pluto_updated = pluto_updated.reset_index()

In [28]:
pluto_updated = pluto_updated.drop('index',axis=1)

In [29]:
fv = fv.reset_index()
fv = fv.drop('index',axis=1)

In [30]:
pluto_updated['BBL'] = pluto_updated['BBL'].astype(str)
fv['BBL'] = fv['BBL'].astype(float)
fv['BBL'] = fv['BBL'].astype(int)
fv['BBL'] = fv['BBL'].astype(str)

In [31]:
pluto_updated['BBL'].head()

0    5007470028
1    5006230252
2    5007130017
3    5007140091
4    5011840039
Name: BBL, dtype: object

In [32]:
fv['BBL'].head()

0    5007470028
1    5007130017
2    5007140091
3    5011840039
4    5007650033
Name: BBL, dtype: object

In [33]:
av_shp = pd.merge(pluto_updated, fv, on='BBL', how='right')

In [34]:
#test = []
#for i in range(0,len(av_shp)):
    #if type(av_shp['geometry'][i])!=type(av_shp['geometry'][0]):
        #test.append(zeros['geometry'][i])

In [35]:
print(av_shp.crs)
print(type(av_shp))

{'init': u'epsg:4326'}
<class 'geopandas.geodataframe.GeoDataFrame'>


In [36]:
print(len(av_shp))

756433


In [37]:
print(type(av_shp))

<class 'geopandas.geodataframe.GeoDataFrame'>


In [38]:
av_shp.head(2)

,BBL,geometry,NumBldgs,NumFloors,ResArea,AssessSqft,ResArea_PerUnit,BuildingAge,Sum_season_vio,Avg_season_vio
0,5007470028,POLYGON ((-74.14250944979312 40.60980609846242...,1,1.0,1488,15.725806,744.0,57,0.0,0.0
1,5007130017,POLYGON ((-74.12595312726884 40.60946755523582...,2,2.5,1316,17.174772,1316.0,97,0.0,0.0


In [39]:
av_shp['Sum_season_vio'] = av_shp['Sum_season_vio'].astype(int)

In [40]:
av_shp = gpd.read_file('../data_processed/bbl_vios_sum_season.shp')

In [41]:
av_shp.head()

,AssessSqft,Avg_season,BBL,BuildingAg,NumBldgs,NumFloors,ResArea,ResArea_Pe,Sum_season,geometry
0,15.725806,0.0,5007470028,57,1,1.0,1488,744.0,0,POLYGON ((-74.14250944979312 40.60980609846242...
1,17.174772,0.0,5007130017,97,2,2.5,1316,1316.0,0,POLYGON ((-74.12595312726884 40.60946755523582...
2,13.135283,0.0,5007140091,42,1,2.0,2188,1094.0,0,POLYGON ((-74.12313095467215 40.60801121850653...
3,9.246656,0.0,5011840039,52,1,2.0,2392,2392.0,0,POLYGON ((-74.15227503511518 40.62802680723158...
4,17.458111,0.0,5007650033,92,2,2.5,1122,1122.0,0,POLYGON ((-74.13032190538277 40.60753648104292...


In [42]:
av_shp = av_shp.rename(columns={'Sum_season':'Sum_season_vio','Avg_season':'Avg_season_vio', \
                               'BuildingAg':'BuildingAge', 'ResArea_Pe':'ResArea_PerUnit'})

#### linear model predicting total volume of seasonal heat and hot water violations

In [43]:
mod = smf.ols(formula='Sum_season_vio ~ NumBldgs + NumFloors + ResArea + AssessSqft + ResArea_PerUnit + BuildingAge'\
              , data=av_shp)
res = mod.fit()
print res.summary()

                            OLS Regression Results                            
Dep. Variable:         Sum_season_vio   R-squared:                       0.021
Model:                            OLS   Adj. R-squared:                  0.021
Method:                 Least Squares   F-statistic:                     2651.
Date:                Sat, 15 Apr 2017   Prob (F-statistic):               0.00
Time:                        16:19:22   Log-Likelihood:            -8.9470e+05
No. Observations:              756433   AIC:                         1.789e+06
Df Residuals:                  756426   BIC:                         1.789e+06
Df Model:                           6                                         
Covariance Type:            nonrobust                                         
                      coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------
Intercept          -0.0634      0.004    -

#### linear model predicting average seasonal heat and hot water violations

In [44]:
mod = smf.ols(formula='Avg_season_vio ~ NumBldgs + NumFloors + ResArea + AssessSqft + ResArea_PerUnit + BuildingAge'\
              , data=av_shp)
res = mod.fit()
print res.summary()

                            OLS Regression Results                            
Dep. Variable:         Avg_season_vio   R-squared:                       0.021
Model:                            OLS   Adj. R-squared:                  0.021
Method:                 Least Squares   F-statistic:                     2651.
Date:                Sat, 15 Apr 2017   Prob (F-statistic):               0.00
Time:                        16:19:23   Log-Likelihood:             1.5394e+05
No. Observations:              756433   AIC:                        -3.079e+05
Df Residuals:                  756426   BIC:                        -3.078e+05
Df Model:                           6                                         
Covariance Type:            nonrobust                                         
                      coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------
Intercept          -0.0158      0.001    -

#### import boilers dataset

In [45]:
boilers = pd.read_csv('../data_processed/boilers.csv')
boilers = boilers.drop('Unnamed: 0',axis=1)

In [46]:
boilers.head(2)

,BBL,avg_consumption,avg_boiler_age,avg_boiler_capacity
0,1000087501,69806.0,26.0,17.92
1,1000130001,45900.0,47.0,16.80


In [47]:
print av_shp['BBL'].dtypes
print boilers['BBL'].dtypes

object
int64


In [48]:
av_shp['BBL'] = av_shp['BBL'].astype(int)

In [49]:
boilers = pd.merge(boilers, av_shp, on='BBL')

In [50]:
len(boilers)

6296

In [51]:
boilers.columns

Index([                u'BBL',     u'avg_consumption',      u'avg_boiler_age',
       u'avg_boiler_capacity',          u'AssessSqft',      u'Avg_season_vio',
               u'BuildingAge',            u'NumBldgs',           u'NumFloors',
                   u'ResArea',     u'ResArea_PerUnit',      u'Sum_season_vio',
                  u'geometry'],
      dtype='object')

#### predict average seasonal heat and hot water violations with boiler characteristics 

In [52]:
mod = smf.ols(formula='Avg_season_vio ~ NumBldgs + NumFloors + ResArea + AssessSqft + ResArea_PerUnit + avg_consumption + avg_boiler_age + avg_boiler_capacity'\
              , data=boilers)
res = mod.fit()
print res.summary()

                            OLS Regression Results                            
Dep. Variable:         Avg_season_vio   R-squared:                       0.037
Model:                            OLS   Adj. R-squared:                  0.036
Method:                 Least Squares   F-statistic:                     30.43
Date:                Sat, 15 Apr 2017   Prob (F-statistic):           3.81e-47
Time:                        16:19:24   Log-Likelihood:                -8187.6
No. Observations:                6296   AIC:                         1.639e+04
Df Residuals:                    6287   BIC:                         1.645e+04
Df Model:                           8                                         
Covariance Type:            nonrobust                                         
                          coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------------------------------
Intercept               0.6738    

#### predict total seasonal heat and hot water violations with boiler characteristics

In [53]:
mod = smf.ols(formula='Sum_season_vio ~ NumBldgs + NumFloors + ResArea + AssessSqft + ResArea_PerUnit + avg_consumption + avg_boiler_age + avg_boiler_capacity'\
              , data=boilers)
res = mod.fit()
print res.summary()

                            OLS Regression Results                            
Dep. Variable:         Sum_season_vio   R-squared:                       0.037
Model:                            OLS   Adj. R-squared:                  0.036
Method:                 Least Squares   F-statistic:                     30.43
Date:                Sat, 15 Apr 2017   Prob (F-statistic):           3.81e-47
Time:                        16:19:24   Log-Likelihood:                -16916.
No. Observations:                6296   AIC:                         3.385e+04
Df Residuals:                    6287   BIC:                         3.391e+04
Df Model:                           8                                         
Covariance Type:            nonrobust                                         
                          coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------------------------------
Intercept               2.6950    

#### try linear model on bbls with violations

In [54]:
non_zeros = av_shp[av_shp['Avg_season_vio'] > 0]

In [55]:
mod = smf.ols(formula='Avg_season_vio ~ NumBldgs + NumFloors + ResArea + AssessSqft + ResArea_PerUnit + BuildingAge'\
              , data=non_zeros)
res = mod.fit()
print res.summary()

                            OLS Regression Results                            
Dep. Variable:         Avg_season_vio   R-squared:                       0.014
Model:                            OLS   Adj. R-squared:                  0.014
Method:                 Least Squares   F-statistic:                     41.58
Date:                Sat, 15 Apr 2017   Prob (F-statistic):           1.27e-50
Time:                        16:19:25   Log-Likelihood:                -25152.
No. Observations:               17234   AIC:                         5.032e+04
Df Residuals:                   17227   BIC:                         5.037e+04
Df Model:                           6                                         
Covariance Type:            nonrobust                                         
                      coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------
Intercept           0.4090      0.035     

#### try linear model on bbls with violations, including boiler characteristics

In [56]:
non_zeros_boiler = boilers[boilers['Avg_season_vio']>0]

In [57]:
mod = smf.ols(formula='Avg_season_vio ~ NumBldgs + NumFloors + ResArea + AssessSqft + ResArea_PerUnit + avg_consumption + avg_boiler_age + avg_boiler_capacity'\
              , data=non_zeros_boiler)
res = mod.fit()
print res.summary()

                            OLS Regression Results                            
Dep. Variable:         Avg_season_vio   R-squared:                       0.022
Model:                            OLS   Adj. R-squared:                  0.019
Method:                 Least Squares   F-statistic:                     6.039
Date:                Sat, 15 Apr 2017   Prob (F-statistic):           1.06e-07
Time:                        16:19:25   Log-Likelihood:                -3576.2
No. Observations:                2136   AIC:                             7170.
Df Residuals:                    2127   BIC:                             7221.
Df Model:                           8                                         
Covariance Type:            nonrobust                                         
                          coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------------------------------
Intercept               1.4037    

#### create new df from log transformations identified in notebook 'mappluto_data_distribution_331'

In [58]:
### transformed data
transformed = pd.DataFrame()
transformed['BBL'] = av_shp['BBL']
transformed['Avg_season_vio'] = av_shp['Avg_season_vio']
transformed['Sum_season_vio'] = av_shp['Sum_season_vio']
#transformed['BuiltFAR'] = np.log(a['BuiltFAR'])
transformed['NumBldgs'] = np.log(av_shp['NumBldgs'])
transformed['NumFloors'] = np.log(av_shp['NumFloors'])
transformed['ResArea'] = np.log(av_shp['ResArea'])
#transformed['UnitsRes'] = np.log(a['UnitsRes'])
transformed['BuildingAge'] = av_shp['BuildingAge']
transformed['AssessSqft'] = np.log(av_shp['AssessSqft'])
#transformed['AssessLand'] = np.log(a['AssessLand'])
transformed['ResArea_PerUnit'] = np.log(av_shp['ResArea_PerUnit'])
transformed['Avg_season_vio_normed'] = transformed['Avg_season_vio']/av_shp['NumBldgs']

#### linear model on transformed data

In [59]:
mod = smf.ols(formula='Avg_season_vio ~ NumBldgs + NumFloors + ResArea + AssessSqft + ResArea_PerUnit + BuildingAge'\
              , data=transformed)
res = mod.fit()
print res.summary()

                            OLS Regression Results                            
Dep. Variable:         Avg_season_vio   R-squared:                       0.052
Model:                            OLS   Adj. R-squared:                  0.052
Method:                 Least Squares   F-statistic:                     6968.
Date:                Sat, 15 Apr 2017   Prob (F-statistic):               0.00
Time:                        16:19:26   Log-Likelihood:             1.6642e+05
No. Observations:              756433   AIC:                        -3.328e+05
Df Residuals:                  756426   BIC:                        -3.327e+05
Df Model:                           6                                         
Covariance Type:            nonrobust                                         
                      coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------
Intercept          -0.1702      0.005    -

In [60]:
mod = smf.ols(formula='Sum_season_vio ~ NumBldgs + NumFloors + ResArea + AssessSqft + ResArea_PerUnit + BuildingAge'\
              , data=transformed)
res = mod.fit()
print res.summary()

                            OLS Regression Results                            
Dep. Variable:         Sum_season_vio   R-squared:                       0.052
Model:                            OLS   Adj. R-squared:                  0.052
Method:                 Least Squares   F-statistic:                     6968.
Date:                Sat, 15 Apr 2017   Prob (F-statistic):               0.00
Time:                        16:19:27   Log-Likelihood:            -8.8222e+05
No. Observations:              756433   AIC:                         1.764e+06
Df Residuals:                  756426   BIC:                         1.765e+06
Df Model:                           6                                         
Covariance Type:            nonrobust                                         
                      coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------
Intercept          -0.6807      0.020    -

#### linear model on transformed data and normalized violation dependent variable by # of buildings, which was then dropped from the feature space

In [61]:
mod = smf.ols(formula='Avg_season_vio_normed ~ NumFloors + ResArea + AssessSqft + ResArea_PerUnit + BuildingAge'\
              , data=transformed)
res = mod.fit()
print res.summary()

                              OLS Regression Results                             
Dep. Variable:     Avg_season_vio_normed   R-squared:                       0.050
Model:                               OLS   Adj. R-squared:                  0.050
Method:                    Least Squares   F-statistic:                     7905.
Date:                   Sat, 15 Apr 2017   Prob (F-statistic):               0.00
Time:                           16:19:27   Log-Likelihood:             2.1861e+05
No. Observations:                 756433   AIC:                        -4.372e+05
Df Residuals:                     756427   BIC:                        -4.371e+05
Df Model:                              5                                         
Covariance Type:               nonrobust                                         
                      coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------
Intercept   

#### check the percentage of bbls which had at least one violation

In [62]:
print len(transformed[transformed['Avg_season_vio']>0])
print len(transformed[transformed['Avg_season_vio']<1])/float(len(transformed))

17234
0.994040450377


In [63]:
print len(av_shp[av_shp['Avg_season_vio']>0])
print len(av_shp[av_shp['Avg_season_vio']<1])/float(len(transformed))

17234
0.994040450377


#### create new dependent variable, which captures violation/no violation as binary options (to run logistic regression)

In [64]:
transformed['vio_binary'] = [1 if x > 0 else 0 for x in transformed['Avg_season_vio']]

In [65]:
transformed[transformed['Avg_season_vio']>0].head()

,BBL,Avg_season_vio,Sum_season_vio,NumBldgs,NumFloors,ResArea,BuildingAge,AssessSqft,ResArea_PerUnit,Avg_season_vio_normed,vio_binary
36,5012780045,0.75,3,0.000000,0.916291,7.408531,32,2.145400,6.715383,0.75,1
151,5012460037,0.50,2,0.000000,0.693147,7.901007,14,2.189790,7.207860,0.50,1
462,5003520078,0.25,1,0.000000,0.693147,7.103322,47,2.794499,6.410175,0.25,1
468,5002090055,1.00,4,0.693147,1.098612,7.396335,97,1.660731,6.703188,0.50,1
715,5008970017,0.25,1,0.000000,1.098612,8.342840,77,3.603853,6.263398,0.25,1


#### run logistic regression on transformed variables and new binary outcome variable

In [66]:
X = transformed[['NumBldgs', 'NumFloors', 'ResArea', 'BuildingAge', 'AssessSqft', 'ResArea_PerUnit']]
y = transformed['vio_binary']

In [67]:
mod = smf.Logit(y, X)
res = mod.fit()
print res.summary()

Optimization terminated successfully.
         Current function value: 0.079096
         Iterations 9
                           Logit Regression Results                           
Dep. Variable:             vio_binary   No. Observations:               756433
Model:                          Logit   Df Residuals:                   756427
Method:                           MLE   Df Model:                            5
Date:                Sat, 15 Apr 2017   Pseudo R-squ.:                  0.2722
Time:                        16:19:31   Log-Likelihood:                -59831.
converged:                       True   LL-Null:                       -82210.
                                        LLR p-value:                     0.000
                      coef    std err          z      P>|z|      [0.025      0.975]
-----------------------------------------------------------------------------------
NumBldgs           -0.7615      0.027    -28.511      0.000      -0.814      -0.709
NumFloors     

In [68]:
res.prsquared

0.27222397464512949

In [69]:
results = pd.DataFrame()
results['y_act'] = transformed['vio_binary']
results['y_pred'] = res.predict(X)
results['y_pred_binary'] = [1 if x > 0.5 else 0 for x in res.predict(X)]

In [70]:
results.head()

,y_act,y_pred,y_pred_binary
0,0,0.010990,0
1,0,0.002758,0
2,0,0.006926,0
3,0,0.002524,0
4,0,0.002815,0


In [71]:
is_acc = []
for i in range(1,10):
    mod = smf.Logit(y, X)
    res = mod.fit()
    is_acc.append(res.prsquared)
print sum(is_acc)/float(len(is_acc))

Optimization terminated successfully.
         Current function value: 0.079096
         Iterations 9
Optimization terminated successfully.
         Current function value: 0.079096
         Iterations 9
Optimization terminated successfully.
         Current function value: 0.079096
         Iterations 9
Optimization terminated successfully.
         Current function value: 0.079096
         Iterations 9
Optimization terminated successfully.
         Current function value: 0.079096
         Iterations 9
Optimization terminated successfully.
         Current function value: 0.079096
         Iterations 9
Optimization terminated successfully.
         Current function value: 0.079096
         Iterations 9
Optimization terminated successfully.
         Current function value: 0.079096
         Iterations 9
Optimization terminated successfully.
         Current function value: 0.079096
         Iterations 9
0.272223974645


#### use cohen's statistic to identify predictive accuracy of model as it compares to randomly selecting values
https://stats.stackexchange.com/questions/82162/cohens-kappa-in-plain-english

In [72]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)
mod = smf.Logit(y_train, X_train)
res = mod.fit()
results=pd.DataFrame()
results['y_act'] = y_test
y_pred = res.predict(X_test)
results['y_pred_binary'] = [1 if x >= 0.5 else 0 for x in y_pred]

Optimization terminated successfully.
         Current function value: 0.078527
         Iterations 9


In [73]:
results.head()

,y_act,y_pred_binary
476010,0,0
404830,1,0
318659,0,0
313593,0,0
353032,0,0


In [74]:
confusion_matrix(results['y_act'], results['y_pred_binary'])

array([[243273,    592],
       [  5429,    329]])

In [75]:
cohen_kappa_score(results['y_act'], results['y_pred_binary'])

0.092746171019537083

#### improve the model accuracy by undersampling the majority class
http://www.marcoaltini.com/blog/dealing-with-imbalanced-data-undersampling-oversampling-and-proper-cross-validation

In [76]:
is_rsquared = []
cohen = []
for i in range(0,10):
    zeros = transformed[transformed['vio_binary']==0]
    sample_zeros = zeros.sample(n=17234)
    newdf = pd.concat([transformed[transformed['vio_binary']==1], sample_zeros])
    X = newdf[['NumBldgs', 'NumFloors', 'ResArea', 'BuildingAge', 'AssessSqft', 'ResArea_PerUnit']]
    y = newdf['vio_binary']
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)
    mod = smf.Logit(y_train, X_train)
    res = mod.fit()
    is_rsquared.append(res.prsquared)
    results=pd.DataFrame()
    results['y_act'] = y_test
    y_pred = res.predict(X_test)
    results['y_pred_binary'] = [1 if x >= 0.5 else 0 for x in y_pred]
    cohen.append(cohen_kappa_score(results['y_act'], results['y_pred_binary']))
print(sum(is_rsquared)/float(len(is_rsquared)))
print(sum(cohen)/float(len(cohen)))

Optimization terminated successfully.
         Current function value: 0.415092
         Iterations 7
Optimization terminated successfully.
         Current function value: 0.416465
         Iterations 7
Optimization terminated successfully.
         Current function value: 0.420586
         Iterations 7
Optimization terminated successfully.
         Current function value: 0.415293
         Iterations 7
Optimization terminated successfully.
         Current function value: 0.420261
         Iterations 7
Optimization terminated successfully.
         Current function value: 0.419586
         Iterations 7
Optimization terminated successfully.
         Current function value: 0.415959
         Iterations 7
Optimization terminated successfully.
         Current function value: 0.421159
         Iterations 7
Optimization terminated successfully.
         Current function value: 0.421375
         Iterations 7
Optimization terminated successfully.
         Current function value: 0.415577
  

In [78]:
is_rsquared = []
for i in range(0,10):
    zeros = transformed[transformed['Avg_season_vio_normed']==0]
    sample_zeros = zeros.sample(n=17234)
    newdf = pd.concat([transformed[transformed['Avg_season_vio_normed']>0], sample_zeros])
    X = newdf[['NumBldgs', 'NumFloors', 'ResArea', 'BuildingAge', 'AssessSqft', 'ResArea_PerUnit']]
    y = newdf['Avg_season_vio_normed']
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)
    train = pd.concat([X_train, y_train],axis=1)
    mod = smf.ols(formula='Avg_season_vio_normed ~ NumFloors + ResArea + AssessSqft + ResArea_PerUnit + BuildingAge'\
              , data=train)
    res = mod.fit()
    is_rsquared.append(res.rsquared)
    results=pd.DataFrame()
    results['y_act'] = y_test
    y_pred = res.predict(X_test)
    results['y_pred_binary'] = [1 if x >= 0.5 else 0 for x in y_pred]
print(sum(is_rsquared)/float(len(is_rsquared)))

0.161100709115


#### oversample the minority class - must finish

In [ ]:
X = transformed[['NumBldgs', 'NumFloors', 'ResArea', 'BuildingAge', 'AssessSqft', 'ResArea_PerUnit']]
y = transformed['vio_binary']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)
X_train_1, X_vali, y_train_1, y_vali = train_test_split(X_train, y_train, test_size=0.33, random_state=42)